In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import r2_score, mean_absolute_error
import pandas as pd
from sqlalchemy import create_engine

# Credentials to connect to the database
username = "username"
password = "DB_password"
hostname = "DB_host"
dbname = "DB_name"

# Split the training and validation datasets using the valid_fraction
def get_data_splits(dataframe, valid_fraction=0.2):
    valid_size = int(len(dataframe) * valid_fraction)
    valid_size = max(valid_size, 1)
    train = dataframe[:-valid_size]
    valid = dataframe[-valid_size:]
    return train, valid

def neural_network(nodes, input_length):
    """
    Create the neural network
    """
    model = Sequential()
    model.add(Dense(nodes, input_dim=input_length, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal', activation='relu'))
    model.compile(loss='mean_absolute_error', optimizer='adam', metrics=["mae"])
    return model

# Create training and validation datasets
def create_train_valid_set():
    # Connect to the database of the e-shop
    engine = create_engine(f"mysql+mysqlconnector://{username}:{password}@{hostname}/{dbname}")

    nn_data = pd.read_sql_table("nn_data", engine)
    nn_data = nn_data.drop(columns=["week", "product_cost", "product_max_bound"])

    train = pd.DataFrame(columns=nn_data.columns)
    valid = pd.DataFrame(columns=nn_data.columns)

    for product in nn_data.product_id.unique():
        dataframe = nn_data[nn_data.product_id == product]
        std = dataframe.iloc[:, -1].std()
        mean = dataframe.iloc[:, -1].mean()
        if std <= mean:
            temp_train, temp_valid = get_data_splits(dataframe)
            train = pd.concat([train, temp_train], ignore_index=True)
            valid = pd.concat([valid, temp_valid], ignore_index=True)

    X_train = train.iloc[:, :-1]
    y_train = train.iloc[:, -1]
    X_valid = valid.iloc[:, :-1]
    y_valid = valid.iloc[:, -1]

    product_encoder = LabelEncoder()
    X_train["product_id"] = product_encoder.fit_transform(X_train["product_id"].astype(str))
    X_valid["product_id"] = product_encoder.transform(X_valid["product_id"].astype(str))

    return X_train, X_valid, y_train, y_valid

# Test the neural network and its performance
def nn_testing():
    X_train, X_valid, y_train, y_valid = create_train_valid_set()

    model = neural_network(23, X_train.shape[1])
    history = model.fit(X_train, y_train,
                        epochs=100, batch_size=256,
                        validation_data=(X_valid, y_valid),
                        verbose=0)

    loss_values = history.history['loss']
    val_loss_values = history.history['val_loss']

    plt.plot(loss_values, label='Training Loss')
    plt.plot(val_loss_values, label='Validation Loss')
    plt.legend()
    plt.show()

    y_train_pred = np.round(model.predict(X_train))
    y_valid_pred = np.round(model.predict(X_valid))
    train_mae = mean_absolute_error(y_train, y_train_pred)
    valid_mae = mean_absolute_error(y_valid, y_valid_pred)

    print(f"The R2 score on the Train set is: {r2_score(y_train, y_train_pred):.3f}")
    print(f"The R2 score on the Valid set is: {r2_score(y_valid, y_valid_pred):.3f}")
    print(f"The MAE on the Train set is: {train_mae:.3f}")
    print(f"The mean of the Train set is: {y_train.mean():.3f}")
    print(f"The percentage of MAE on Train set is: {(train_mae / y_train.mean()) * 100:.2f}%")
    print(f"The MAE on the Valid set is: {valid_mae:.3f}")
    print(f"The mean of the Valid set is: {y_valid.mean():.3f}")
    print(f"The percentage of MAE on Valid set is: {(valid_mae / y_valid.mean()) * 100:.2f}%")

# Final training of the neural network
def nn_final_training():
    engine = create_engine(f"mysql+mysqlconnector://{username}:{password}@{hostname}/{dbname}")

    nn_data = pd.read_sql_table("nn_data", engine)
    nn_data = nn_data.drop(columns=["week", "product_cost", "product_max_bound"])

    X_train = nn_data.iloc[:, :-1]
    y_train = nn_data.iloc[:, -1]

    product_encoder = LabelEncoder()
    X_train["product_id"] = product_encoder.fit_transform(X_train["product_id"].astype(str))

    model = neural_network(23, X_train.shape[1])
    model.fit(X_train, y_train, epochs=50, batch_size=16, verbose=0)
    model.save("final_model.h5")
    return product_encoder

if __name__ == "__main__":
    nn_testing()
